In [1]:
import pandas as pd


https://habr.com/ru/companies/otus/articles/755772/

In [2]:
# !kaggle datasets download crowdflower/twitter-user-gender-classification

In [3]:
df = pd.read_csv('gender-classifier-DFE-791531.csv', encoding="latin1")

In [4]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [5]:
df.text

0        Robbie E Responds To Critics After Win Against...
1        ÛÏIt felt like they were my friends and I was...
2        i absolutely adore when louis starts the songs...
3        Hi @JordanSpieth - Looking at the url - do you...
4        Watching Neighbours on Sky+ catching up with t...
                               ...                        
20045    @lookupondeath ...Fine, and I'll drink tea too...
20046    Greg Hardy you a good player and all but don't...
20047    You can miss people and still never want to se...
20048    @bitemyapp i had noticed your tendency to pee ...
20049    I think for my APUSH creative project I'm goin...
Name: text, Length: 20050, dtype: object

In [91]:
df.gender.isna().sum()

np.int64(0)

In [92]:
df = df[df.gender.notna()]

In [93]:
from sklearn import preprocessing

enc = preprocessing.OrdinalEncoder()
enc.fit(df[['gender']])

OrdinalEncoder()

In [94]:
df['gender_num'] = enc.transform(df[['gender']])

In [95]:
import re


def remove_link(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text, flags=re.MULTILINE)


def remove_person(text):
    return re.sub(r'@\S+', '', text, flags=re.MULTILINE)


def remove_special(text):
    return re.sub('[^A-Za-z0-9.]+', ' ', text)



In [96]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Загрузка стоп-слов и пунктуации
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)


def preprocess_text(text):
    words = word_tokenize(text.lower())  # Привести к нижнему регистру и токенизировать
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k142\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k142\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\k142\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [97]:
# !python -m spacy download en_core_web_sm

In [98]:
import spacy

# Загрузка языковой модели spaCy
nlp = spacy.load("en_core_web_sm")


# nlp = spacy.load("ru_core_news_sm")

def preprocess_and_lemmatize(text):
    doc = nlp(text.lower())  # Привести к нижнему регистру и лемматизировать
    lemmatized_words = [token.lemma_ for token in doc if token.text not in punctuation and token.text not in stop_words]
    return " ".join(lemmatized_words)

In [99]:
def preprocess(text):
    text = remove_link(text)
    text = remove_person(text)
    text = remove_special(text)
    text = preprocess_text(text)
    return preprocess_and_lemmatize(text)


In [100]:
text = df.text.iloc[3]

In [101]:
text

"Hi @JordanSpieth - Looking at the url - do you use @IFTTT?!  Don't typically see an advanced user on the @PGATOUR! https://t.co/H68ou5PE9L"

In [102]:
preprocess(text)

'hi look url use typically see advanced user'

In [103]:
df['text1'] = df.text.apply(preprocess)

In [104]:
df[['text', 'text1']]

,text,text1
0,Robbie E Responds To Critics After Win Against Eddie Edwards In The #WorldTitleSeries https://t.co/NSybBmVjKZ,robbie e respond critic win eddie edwards worldtitleseries
1,ÛÏIt felt like they were my friends and I was living the story with themÛ https://t.co/arngE0YHNO #retired #IAN1 https://t.co/CIzCANPQFz,feel like friend living story retire ian1
2,i absolutely adore when louis starts the songs it hits me hard but it feels good,absolutely adore louis starts song hit hard feel good
3,Hi @JordanSpieth - Looking at the url - do you use @IFTTT?! Don't typically see an advanced user on the @PGATOUR! https://t.co/H68ou5PE9L,hi look url use typically see advanced user
4,Watching Neighbours on Sky+ catching up with the Neighbs!! Xxx _Ù÷Ä_Ù÷Ä_Ù÷Ä_ÙÔÎ_ÙÈ_ÙÔ_ÙÈ Xxx,watch neighbour sky catch neighbs xxx xxx
...,...,...
20045,"@lookupondeath ...Fine, and I'll drink tea too. I love you.",... fine drink tea love
20046,Greg Hardy you a good player and all but don't get in the face of Dez Bryant. Learn your place mane.,greg hardy good player get face dez bryant learn place mane
20047,You can miss people and still never want to see them again,miss people still never want see
20048,"@bitemyapp i had noticed your tendency to pee on the carpet, but i didn't want to say anything :P",notice tendency pee carpet want say anything p


In [105]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [106]:
X_train, X_test, y_train, y_test = train_test_split(df.text1, df.gender_num, test_size=0.2, random_state=0)

In [120]:
df.shape

(19953, 28)

In [121]:
X_train.shape

(15962,)

In [123]:
X_test.shape

(3991,)

In [107]:
y_train.value_counts()

gender_num
1.0    5365
2.0    4928
0.0    4761
3.0     908
Name: count, dtype: int64

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [109]:
# Преобразование текстовых данных в матрицу TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [110]:
X_test_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 28437 stored elements and shape (3991, 19704)>

In [111]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

In [112]:
# Предсказание категорий для тестовых данных
y_pred = classifier.predict(X_test_tfidf)

In [113]:
# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

Точность модели: 0.5031320471059885


In [115]:
from sklearn.linear_model import SGDClassifier

classifier=SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42)

In [116]:
classifier.fit(X_train_tfidf, y_train)

SGDClassifier(alpha=0.001, random_state=42)

In [117]:
y_pred = classifier.predict(X_test_tfidf)


In [118]:
y_pred

array([0., 2., 2., ..., 1., 1., 1.], shape=(3991,))

In [119]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

Точность модели: 0.5011275369581558


In [125]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.56      0.59      0.57      1181
         1.0       0.48      0.68      0.56      1335
         2.0       0.47      0.31      0.37      1266
         3.0       0.00      0.00      0.00       209

    accuracy                           0.50      3991
   macro avg       0.38      0.40      0.38      3991
weighted avg       0.47      0.50      0.48      3991

